In [16]:
import pandas as pd
import numpy as np
import datetime as dt

In [17]:
# read in csv file with glassdoor data collected from webscraper
df = pd.read_csv("glassdoor_jobs.csv")

In [18]:
# split company_name into company and rating a columns
df[["company", "rating"]] = df["company_name"].str.split("\n", expand=True)
df.drop(columns="company_name", inplace=True)
df = df.iloc[:, [11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
df

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue
0,House of Bèta,4.4,Amsterdam,Traineeship Data Science,€ 2.726 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,—Dutch speaking and writing proficiency is req...,201 tot 500 werknemers,2019,Beursgenoteerd bedrijf,IT-ondersteuning,IT,Onbekend/niet van toepassing
1,Politie,3.8,Driebergen,data engineer Multi Data Center (MDC),€ 4.283 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Criminele netwerken infiltreren onze wijken en...,1001 tot 5000 werknemers,Overheid,Rijksoverheid,Openbaar bestuur,Onbekend/niet van toepassing,NaN
2,Breinstein Detachering,4.5,Amsterdam,Junior Data Engineer,€ 3.600 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,Functieomschrijving\nOrganisaties beschikken o...,51 tot 200 werknemers,2010,Privébedrijf,HR-advies,HR en personeel,Onbekend/niet van toepassing
3,ABN AMRO,3.9,Amstelveen,Data Scientist (Security),NaN,€ 3K - € 7K (Schatting werkgever),Data Scientist (Security)\nAt a glance\nWe are...,10000+ medewerkers,1991,Overheid,Banken en kredietverstrekking,Finance,5-10 miljard USD
4,Ministerie van Defensie,4.2,Den Haag,Data-analist,€ 5.450 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K,Wat ga je doen?\nAls data-analist bij de Inspe...,10000+ medewerkers,Overheid,Aerospace en defensie,Aerospace en defensie,Onbekend/niet van toepassing,NaN


In [19]:
# remove rows with empty values
df = df.dropna()
df

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue
0,House of Bèta,4.4,Amsterdam,Traineeship Data Science,€ 2.726 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,—Dutch speaking and writing proficiency is req...,201 tot 500 werknemers,2019,Beursgenoteerd bedrijf,IT-ondersteuning,IT,Onbekend/niet van toepassing
2,Breinstein Detachering,4.5,Amsterdam,Junior Data Engineer,€ 3.600 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,Functieomschrijving\nOrganisaties beschikken o...,51 tot 200 werknemers,2010,Privébedrijf,HR-advies,HR en personeel,Onbekend/niet van toepassing


In [20]:
# add indicator for salary provided hourly
df["hourly"] = df["salary_range"].map(lambda x: 1 if "per uur" in x.lower() else 0)

# clean up salary range
salary_by_employee = df["salary_range"].map(lambda x: x.split(":")[1])
salary_by_employee_blank = salary_by_employee.map(lambda x: x.replace("€", "").replace("K",""))
salary_by_employee_blank_hr = salary_by_employee_blank.map(lambda x: x.lower().replace("per uur", ""))

# # add min, max and avg salaries
df[["min_salary", "max_salary"]] = salary_by_employee_blank_hr.str.split(" - ", expand=True)
df.loc[df['max_salary'].isna(), "max_salary"] = df["min_salary"]
df["min_salary"] = df["min_salary"].map(lambda x: float(x.replace(",", ".")))
df["max_salary"] = df["max_salary"].map(lambda x: float(x.replace(",", ".")))
df["avg_salary"] = (df.min_salary + df.max_salary) / 2
df


/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hourly"] = df["salary_range"].map(lambda x: 1 if "per uur" in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["min_salary", "max_salary"]] = salary_by_employee_blank_hr.str.split(" - ", expand=True)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:10: S

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue,hourly,min_salary,max_salary,avg_salary
0,House of Bèta,4.4,Amsterdam,Traineeship Data Science,€ 2.726 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,—Dutch speaking and writing proficiency is req...,201 tot 500 werknemers,2019,Beursgenoteerd bedrijf,IT-ondersteuning,IT,Onbekend/niet van toepassing,0,3.0,3.0,3.0
2,Breinstein Detachering,4.5,Amsterdam,Junior Data Engineer,€ 3.600 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,Functieomschrijving\nOrganisaties beschikken o...,51 tot 200 werknemers,2010,Privébedrijf,HR-advies,HR en personeel,Onbekend/niet van toepassing,0,3.0,5.0,4.0


In [21]:
# check value counts
df.location.value_counts()

Amsterdam    2
Name: location, dtype: int64

In [22]:
# age of company
df["age"] = df["founded"].map(lambda x: dt.date.today().year - int(x))
df

/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/761360882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["age"] = df["founded"].map(lambda x: dt.date.today().year - int(x))


,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue,hourly,min_salary,max_salary,avg_salary,age
0,House of Bèta,4.4,Amsterdam,Traineeship Data Science,€ 2.726 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,—Dutch speaking and writing proficiency is req...,201 tot 500 werknemers,2019,Beursgenoteerd bedrijf,IT-ondersteuning,IT,Onbekend/niet van toepassing,0,3.0,3.0,3.0,3
2,Breinstein Detachering,4.5,Amsterdam,Junior Data Engineer,€ 3.600 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,Functieomschrijving\nOrganisaties beschikken o...,51 tot 200 werknemers,2010,Privébedrijf,HR-advies,HR en personeel,Onbekend/niet van toepassing,0,3.0,5.0,4.0,12


In [33]:
# parse job description
df["python_yn"] = df["job_description"].map(lambda x: 1 if 'python' in x.lower() else 0)
df["r_studio_yn"] = df["job_description"].map(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
df["spark_yn"] = df["job_description"].map(lambda x: 1 if 'spark' in x.lower() else 0)
df["aws_yn"] = df["job_description"].map(lambda x: 1 if 'aws' in x.lower() else 0)
df["excel_yn"] = df["job_description"].map(lambda x: 1 if 'excel' in x.lower() else 0)
df["azure_yn"] = df["job_description"].map(lambda x: 1 if 'azure' in x.lower() else 0)
df["sql_yn"] = df["job_description"].map(lambda x: 1 if 'sql' in x.lower() else 0)

df

/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/541884246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["python_yn"] = df["job_description"].map(lambda x: 1 if 'python' in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/541884246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["r_studio_yn"] = df["job_description"].map(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ip

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,...,max_salary,avg_salary,age,python_yn,r_studio_yn,spark_yn,aws_yn,excel_yn,azure_yn,sql_yn
0,House of Bèta,4.4,Amsterdam,Traineeship Data Science,€ 2.726 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,—Dutch speaking and writing proficiency is req...,201 tot 500 werknemers,2019,Beursgenoteerd bedrijf,...,3.0,3.0,3,0,0,0,0,1,0,1
2,Breinstein Detachering,4.5,Amsterdam,Junior Data Engineer,€ 3.600 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,Functieomschrijving\nOrganisaties beschikken o...,51 tot 200 werknemers,2010,Privébedrijf,...,5.0,4.0,12,1,0,0,1,0,1,1


In [37]:
# write result to csv
df.to_csv("glassdoor_jobs_cleaned.csv", index=False)